In [24]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None
from numpy import loadtxt
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline
import matplotlib
#matplotlib.use('agg')
matplotlib.style.use('ggplot')
import pickle as pkl
from matplotlib import pyplot as plt
from collections import Counter
from functools import reduce
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import random
random.seed(1991)
import glob

In [2]:
novel_compounds=pd.read_csv("/data/dharp/compounding/datasets/novel_compounds.txt",delim_whitespace=True,header=None)
novel_compounds.columns=['modifier','head']
modifiers_list=novel_compounds['modifier'].unique()
heads_list=novel_compounds['head'].unique()

In [5]:
dfm_non_dec=pd.read_csv("/data/dharp/compounding/datasets/dist_sem_non_decadal.csv")
dfm_non_dec

,modifier,head,sim_bw_constituents,sim_with_head,sim_with_modifier,log_ratio,ppmi,local_mi
0,a_n,aaaaa_n,0.033,1.000,0.033,0.000,0.481,0.002
1,a_n,addison_n,0.008,0.609,0.010,0.000,0.330,0.002
2,a_n,anti_n,0.000,0.004,0.086,0.000,0.127,0.002
3,a_n,archive_n,0.001,0.026,0.028,0.000,0.122,0.002
4,a_n,atom_n,0.112,0.616,0.154,0.000,0.000,0.002
5,a_n,b_n,0.546,0.536,0.914,0.001,0.260,0.003
6,a_n,ballet_n,0.000,0.005,0.006,0.000,0.144,0.002
7,a_n,band_n,0.127,0.164,0.035,0.000,0.083,0.002
8,a_n,baseball_n,0.000,0.085,0.003,0.000,0.114,0.002
9,a_n,behavior_n,0.000,0.013,0.026,0.000,0.106,0.002


In [11]:
modifiers=dfm_non_dec.loc[dfm_non_dec.modifier.isin(modifiers_list)].drop(['head','sim_with_head'],axis=1)
modifier_features=modifiers.groupby(['modifier']).agg({'mean','std'})
modifier_features_1=modifier_features.columns.get_level_values(0)
modifier_features_2=modifier_features.columns.get_level_values(1)

cur_year=0
new_columns=[]
for year in modifier_features_1:
    new_columns.append("mod_"+str(year)+"_"+modifier_features_2[cur_year])
    cur_year+=1
modifier_features.columns=new_columns
modifier_features.fillna(modifier_features.mean(),inplace=True)
modifier_features.reset_index(inplace=True)
modifier_features

,modifier,mod_sim_bw_constituents_mean,mod_sim_bw_constituents_std,mod_sim_with_modifier_mean,mod_sim_with_modifier_std,mod_log_ratio_mean,mod_log_ratio_std,mod_ppmi_mean,mod_ppmi_std,mod_local_mi_mean,mod_local_mi_std
0,a_n,0.014,0.046,0.032,0.071,0.000,0.000,0.164,0.136,0.002,0.000
1,aaa_n,0.082,0.232,0.162,0.329,0.000,0.000,0.236,0.156,0.002,0.000
2,aarhus_n,0.000,0.001,0.035,0.019,0.000,0.000,0.274,0.005,0.002,0.000
3,abbreviated_n,0.081,0.192,0.364,0.316,0.000,0.000,0.371,0.203,0.002,0.000
4,abduction_n,0.059,0.099,0.400,0.193,0.000,0.000,0.359,0.132,0.002,0.000
5,abecedarian_n,0.084,0.113,0.987,0.228,0.000,0.000,0.362,0.130,0.002,0.000
6,abilities_n,0.101,0.170,0.218,0.283,0.000,0.000,0.331,0.207,0.002,0.000
7,ability_n,0.051,0.109,0.159,0.226,0.000,0.000,0.138,0.112,0.002,0.000
8,abnormalities_n,0.001,0.113,0.775,0.228,0.000,0.000,0.358,0.130,0.002,0.000
9,abolition_n,0.106,0.178,0.148,0.186,0.000,0.000,0.190,0.134,0.002,0.000


In [12]:
modifier_features.describe()

,mod_sim_bw_constituents_mean,mod_sim_bw_constituents_std,mod_sim_with_modifier_mean,mod_sim_with_modifier_std,mod_log_ratio_mean,mod_log_ratio_std,mod_ppmi_mean,mod_ppmi_std,mod_local_mi_mean,mod_local_mi_std
count,9079.000,9079.000,9079.000,9079.000,9079.000,9079.000,9079.000,9079.000,9079.000,9079.000
mean,0.073,0.113,0.290,0.228,0.000,0.000,0.261,0.130,0.002,0.000
std,0.091,0.086,0.236,0.101,0.000,0.001,0.126,0.039,0.000,0.001
min,0.000,0.000,0.005,0.000,0.000,0.000,0.032,0.000,0.002,0.000
25%,0.021,0.048,0.131,0.163,0.000,0.000,0.169,0.110,0.002,0.000
50%,0.046,0.103,0.206,0.218,0.000,0.000,0.245,0.130,0.002,0.000
75%,0.094,0.157,0.364,0.272,0.000,0.000,0.332,0.148,0.002,0.000
max,1.000,0.667,1.000,0.701,0.015,0.080,0.948,0.466,0.015,0.072


In [13]:
heads=dfm_non_dec.loc[dfm_non_dec['head'].isin(heads_list)].drop(['modifier','sim_with_modifier'],axis=1)
head_features=heads.groupby(['head']).agg({'mean','std'})
head_features_1=head_features.columns.get_level_values(0)
head_features_2=head_features.columns.get_level_values(1)

cur_year=0
new_columns=[]
for year in head_features_1:
    new_columns.append("head_"+str(year)+"_"+head_features_2[cur_year])
    cur_year+=1
head_features.columns=new_columns
head_features.fillna(head_features.mean(),inplace=True)
head_features.reset_index(inplace=True)
head_features

,head,head_sim_bw_constituents_mean,head_sim_bw_constituents_std,head_sim_with_head_mean,head_sim_with_head_std,head_log_ratio_mean,head_log_ratio_std,head_ppmi_mean,head_ppmi_std,head_local_mi_mean,head_local_mi_std
0,a_n,0.057,0.123,0.244,0.316,0.000,0.001,0.109,0.115,0.002,0.001
1,aaron_n,0.056,0.070,0.269,0.256,0.000,0.000,0.338,0.101,0.002,0.000
2,ab_n,0.173,0.228,0.265,0.246,0.000,0.001,0.216,0.151,0.002,0.001
3,abandonment_n,0.015,0.018,0.216,0.167,0.000,0.000,0.308,0.141,0.002,0.000
4,abbreviation_n,0.071,0.120,0.252,0.226,0.000,0.000,0.296,0.143,0.002,0.000
5,abc_n,0.151,0.288,0.159,0.270,0.000,0.000,0.283,0.133,0.002,0.000
6,abduction_n,0.011,0.021,0.231,0.332,0.000,0.000,0.309,0.101,0.002,0.000
7,abdul_n,0.020,0.036,0.284,0.297,0.000,0.000,0.284,0.086,0.002,0.000
8,abdullah_n,0.106,0.147,0.314,0.368,0.000,0.000,0.227,0.204,0.002,0.000
9,ability_n,0.023,0.082,0.092,0.177,0.000,0.000,0.172,0.128,0.002,0.000


## Positive Set

In [20]:
positive_df=pd.merge(novel_compounds,head_features,on=["head"])
positive_df=pd.merge(positive_df,modifier_features,on=["modifier"])
positive_df['Plausibility']=True
positive_df

,modifier,head,head_sim_bw_constituents_mean,head_sim_bw_constituents_std,head_sim_with_head_mean,head_sim_with_head_std,head_log_ratio_mean,head_log_ratio_std,head_ppmi_mean,head_ppmi_std,...,mod_sim_bw_constituents_std,mod_sim_with_modifier_mean,mod_sim_with_modifier_std,mod_log_ratio_mean,mod_log_ratio_std,mod_ppmi_mean,mod_ppmi_std,mod_local_mi_mean,mod_local_mi_std,Plausibility
0,a_n,peaceful_n,0.008,0.010,0.581,0.569,0.000,0.000,0.550,0.072,...,0.046,0.032,0.071,0.000,0.000,0.164,0.136,0.002,0.000,True
1,a_n,robinson_n,0.046,0.093,0.072,0.088,0.000,0.000,0.226,0.125,...,0.046,0.032,0.071,0.000,0.000,0.164,0.136,0.002,0.000,True
2,a_n,supply_n,0.110,0.235,0.094,0.192,0.000,0.004,0.045,0.076,...,0.046,0.032,0.071,0.000,0.000,0.164,0.136,0.002,0.000,True
3,india_n,peaceful_n,0.008,0.010,0.581,0.569,0.000,0.000,0.550,0.072,...,0.072,0.134,0.158,0.000,0.003,0.057,0.094,0.002,0.004,True
4,india_n,standard_n,0.050,0.097,0.058,0.084,0.000,0.001,0.073,0.095,...,0.072,0.134,0.158,0.000,0.003,0.057,0.094,0.002,0.004,True
5,india_n,award_n,0.047,0.112,0.109,0.166,0.000,0.001,0.196,0.158,...,0.072,0.134,0.158,0.000,0.003,0.057,0.094,0.002,0.004,True
6,india_n,joint_n,0.099,0.194,0.153,0.206,0.000,0.001,0.158,0.145,...,0.072,0.134,0.158,0.000,0.003,0.057,0.094,0.002,0.004,True
7,india_n,companion_n,0.069,0.119,0.097,0.135,0.000,0.000,0.219,0.155,...,0.072,0.134,0.158,0.000,0.003,0.057,0.094,0.002,0.004,True
8,india_n,foundation_n,0.018,0.053,0.057,0.125,0.000,0.001,0.137,0.125,...,0.072,0.134,0.158,0.000,0.003,0.057,0.094,0.002,0.004,True
9,india_n,gentleman_n,0.091,0.174,0.144,0.201,0.000,0.001,0.169,0.147,...,0.072,0.134,0.158,0.000,0.003,0.057,0.094,0.002,0.004,True


## Negative Sets

In [15]:
def neg_df_creator(file):
    pkl_file=pkl.load( open(file,'rb'))
    df=pd.DataFrame(pkl_file)
    df.columns=['modifier','head']
    negative_df=pd.merge(df,head_features,on=["head"])
    negative_df=pd.merge(negative_df,modifier_features,on=["modifier"])
    negative_df['Plausibility']=False
    return negative_df

In [16]:
def df_joiner(files):
    df_list=[]
    for file in files:
        neg_df=neg_df_creator(file)
        whole_df=pd.concat([neg_df,positive_df])
        df_list.append(whole_df)
    
    return df_list

In [21]:
corrupt_modifier_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_modifier*"):
    corrupt_modifier_files.append(file)
corrupt_modifiers=df_joiner(corrupt_modifier_files)

In [22]:
corrupt_head_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_head*"):
    corrupt_head_files.append(file)
corrupt_heads=df_joiner(corrupt_head_files)

In [25]:
acc_ch=[]
for i,corrupt_head in enumerate(corrupt_heads):
    data=corrupt_head.drop(['modifier','head'],axis=1)
    X, Y = data.iloc[:,:-1],data.iloc[:,-1]
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1991)
    # fit model no training data
    model = XGBClassifier(njobs=-1)
    model.fit(X_train, y_train)
    # make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    print("DF",i+1)
    accuracy = accuracy_score(y_test, predictions)
    acc_ch.append(accuracy)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))


DF 1
Accuracy: 68.24%
DF 2
Accuracy: 68.51%
DF 3
Accuracy: 68.12%
DF 4
Accuracy: 68.55%
DF 5
Accuracy: 68.56%
DF 6
Accuracy: 68.31%
DF 7
Accuracy: 68.18%
DF 8
Accuracy: 68.32%
DF 9
Accuracy: 69.03%
DF 10
Accuracy: 68.53%


In [27]:
np.mean(acc_ch)

0.6843673110720563

In [28]:
np.std(acc_ch)

0.0025127661977756297

In [26]:
acc_cm=[]
for i,corrupt_head in enumerate(corrupt_modifiers):
    data=corrupt_head.drop(['modifier','head'],axis=1)
    X, Y = data.iloc[:,:-1],data.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1991)
# fit model no training data
    model = XGBClassifier(njobs=-1)
    model.fit(X_train, y_train)
# make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
# evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    acc_cm.append(accuracy)
    print("DF",i+1)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))

DF 1
Accuracy: 66.16%
DF 2
Accuracy: 65.49%
DF 3
Accuracy: 66.40%
DF 4
Accuracy: 65.89%
DF 5
Accuracy: 66.18%
DF 6
Accuracy: 65.94%
DF 7
Accuracy: 66.23%
DF 8
Accuracy: 66.27%
DF 9
Accuracy: 65.57%
DF 10
Accuracy: 66.15%


In [30]:
np.mean(acc_cm)

0.6602724077328647

In [31]:
np.std(acc_cm)

0.0028491742103882137